In [1]:
import pdfplumber
import numpy as np
import pandas as pd
from WindPy import w


In [2]:

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 

w.isconnected() # 判断WindPy是否已经登录成功


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [3]:
#改金额单位
def mc(amount):
    value = ''
    amount = float(amount)
    if abs(amount) >= 10000 and abs(amount) < 100000000:
        val = round(amount/10000,2)
        value = str(val) + "万元"
    elif abs(amount) >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"
        
    return value 

In [4]:
# tables = tabula.read_pdf("20190821-002368.SZ-太极股份：2019年半年度报告.pdf",pages = '6,7') #读取pdf
# code = '002368.SZ'

# with pdfplumber.open("20190821-300085.SZ-银之杰：2019年半年度报告.pdf") as pdf:
#     # 获取第一页
#     page = pdf.pages[5]



#     # 解析表格
#     tables = page.extract_tables()
# #     for table in tables:
# #         print(table)
# #         # df = pd.DataFrame(table[1:], columns=table[0])
# #         for row in table:
# #             for cell in row:
# #                 print(cell, end="\t|")
# #             print()

In [5]:
pdf = pdfplumber.open("20190821-300085.SZ-银之杰：2019年半年度报告.pdf")

code = input("股票代码：")


股票代码：300085.SZ


In [6]:
# 手动输入报表摘要所在页码
# page_num = input("page number:")
# page_num = int(page_num)


In [7]:
# 自动查找页码
counter = 0
for i in range(2,10):
    p0 = pdf.pages[i]
    pp = p0.extract_text(x_tolerance=3, y_tolerance=3)
    if pp.find("主要会计数据")== -1:
        counter = counter + 1
    elif pp.find("主要会计数据")> -1:
        page_num = i
        break
        
        

In [8]:
page_num

7

In [9]:
p0 = pdf.pages[page_num]
table = p0.extract_table()


In [10]:
table

[['', '本报告期', '上年同期', '本报告期比上年同期增减'],
 ['营业总收入（元）', '539,796,493.99', '637,297,654.13', '-15.30%'],
 ['归属于上市公司股东的净利润（元）', '-22,623,049.65', '1,281,561.04', '-1,865.27%'],
 ['归属于上市公司股东的扣除非经常性损', '', '', ''],
 [None, '-34,265,268.05', '-2,207,197.30', '-1,452.43%'],
 ['益后的净利润（元）', None, None, None],
 [None, '', '', ''],
 ['经营活动产生的现金流量净额（元）', '-146,789,281.46', '-91,801,403.24', '-59.90%'],
 ['基本每股收益（元/股）', '-0.0320', '0.0018', '-1,877.78%'],
 ['稀释每股收益（元/股）', '-0.0320', '0.0018', '-1,877.78%'],
 ['加权平均净资产收益率', '-1.67%', '0.10%', '-1.77百分点'],
 ['', '', '', '本报告期末比上年度末增'],
 ['', '本报告期末', '上年度末', None],
 [None, None, None, '减'],
 ['', '', '', None],
 ['总资产（元）', '2,129,339,158.27', '2,154,554,005.56', '-1.17%'],
 ['归属于上市公司股东的净资产（元）', '1,330,550,759.67', '1,351,266,299.89', '-1.53%']]

In [11]:

df = pd.DataFrame(table[1:6], columns=table[0])
# for column in ["Effective", "Received"]:
#     df[column] = df[column].str.replace(" ", "")
df

,,本报告期,上年同期,本报告期比上年同期增减
0,营业总收入（元）,"539,796,493.99","637,297,654.13",-15.30%
1,归属于上市公司股东的净利润（元）,"-22,623,049.65","1,281,561.04","-1,865.27%"
2,归属于上市公司股东的扣除非经常性损,,,
3,None,"-34,265,268.05","-2,207,197.30","-1,452.43%"
4,益后的净利润（元）,None,None,None


In [16]:
data = df.values
percent = {}

#去除逗号
for i in [0,1,3]:
    for j in [1,2,3]:
        percent[i] = data[i,3]
        percent[i] = percent[i].replace('%','')
        data[i,j] = data[i,j].replace(',','')



In [17]:
# 修改数据类型
percent[0] = float(percent[0])
percent[1] = float(percent[1])
percent[3] = float(percent[3])

# 判断增减
updown = {}
for i in [0,1,3]:
    if percent[i]>=0:
        updown[i]="增加"
    else:
        updown[i]="减少"
        percent[i]=abs(percent[i])



In [20]:
percent[3]

1452.43

In [21]:
Revenue = {'obj':data[0,0],'this':data[0,1],'last':data[0,2],'varpct':percent[0]} #把数据写入dict

NetProfit_sh = {'obj':data[1,0],'this':data[1,1],'last':data[1,2],'varpct':percent[1]} #把数据写入dict

NetProfit_sh_loss = {'obj':data[2,0],'this':data[3,1],'last':data[3,2],'varpct':percent[3]} #把数据写入dict


In [22]:
line1_h1 = "20H1营业总收入" + mc(Revenue['this']) + ",上年同期为" + mc(Revenue['last']) + ",同比" + updown[0] + str(percent[0]) + '%'
line2_h1 = "20H1归母净利润" + mc(NetProfit_sh['this']) + ",上年同期为" + mc(NetProfit_sh['last']) + ",同比" + updown[1] + str(percent[1]) + '%'
line3_h1 = "20H1扣非归母净利润" + mc(NetProfit_sh_loss['this']) + ",上年同期为" + mc(NetProfit_sh_loss['last']) + ",同比" + updown[3] + str(percent[3]) + '%'

In [23]:
print(line1_h1)
print(line2_h1)
print(line3_h1)

20H1营业总收入5.4亿元,上年同期为6.37亿元,同比减少15.3%
20H1归母净利润-2262.3万元,上年同期为128.16万元,同比减少1865.27%
20H1扣非归母净利润-3426.53万元,上年同期为-220.72万元,同比减少1452.43%


In [ ]:
###### 以下是wind部分 #######

In [24]:
#营业总收入 _y
# 从wind查找q1数据

wsddata1 = w.wsd(code, "tot_oper_rev", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata1


.ErrorCode=0
.Codes=[300085.SZ]
.Fields=[TOT_OPER_REV]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[255091348.19,539796493.99,843768075.24,1181006553.61,256253083.08]]

In [25]:
#计算q2数据
lastq1_y = wsddata1.Data[0][0] #选取2019年q1数据
lasth1_y = wsddata1.Data[0][1] #选取2019年h1数据
thisq1_y = wsddata1.Data[0][4] #选取2020年q1数据
thish1_y = float(Revenue['this'])
lastq2_y = lasth1_y - lastq1_y    #计算2019年q2数据
thisq2_y = thish1_y - thisq1_y 

#计算同比增长/减少
growth_year = {}
growth_year[0] = (thisq2_y - lastq2_y)/abs(lastq2_y) * 100

In [26]:
#归母净利润 _g
# 从wind查找q1数据
wsddata2 = w.wsd(code, "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata2



.ErrorCode=0
.Codes=[300085.SZ]
.Fields=[NP_BELONGTO_PARCOMSH]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-26021350.11,-22623049.65,1150928.1,4917894.1,-15271142.68]]

In [27]:

#计算q2数据
lastq1_g = wsddata2.Data[0][0] #选取2019年q1数据
lasth1_g = wsddata2.Data[0][1] #选取2019年h1数据
thisq1_g = wsddata2.Data[0][4] #选取2020年q1数据
thish1_g = float(NetProfit_sh['this'])
lastq2_g = lasth1_g - lastq1_g     #计算2019年q2数据
thisq2_g = thish1_g - thisq1_g 

#计算同比增长/减少
growth_year[1] = (thisq2_g - lastq2_g)/abs(lastq2_g) * 100

In [28]:
#扣非归母净利润 _k
# 从wind查找q1数据
wsddata3 = w.wsd(code, "deductedprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata3


.ErrorCode=0
.Codes=[300085.SZ]
.Fields=[DEDUCTEDPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-26855078.55,-34265268.05,-18701231.97,-18185297.38,-18455344.69]]

In [29]:
#计算q2数据
lastq1_k = wsddata3.Data[0][0] #选取2019年q1数据
lasth1_k = wsddata3.Data[0][1] #选取2019年h1数据
thisq1_k = wsddata3.Data[0][4] #选取2020年q1数据
thish1_k = float(NetProfit_sh_loss['this'])
lastq2_k = lasth1_k - lastq1_k     #计算2019年q2数据
thisq2_k = thish1_k - thisq1_k 

#计算同比增长/减少
growth_year[2] = (thisq2_k - lastq2_k)/abs(lastq2_k) * 100

In [30]:
# 判断增减
updown_q2 = {}
for i in [0,1,2]:
    if growth_year[i]>=0:
        updown_q2[i]="增加"
    else:
        updown_q2[i]="减少"
        growth_year[i]=abs(growth_year[i])


In [34]:
line1_q2 = "20Q2营业总收入" + mc(thisq2_y) + ",上年同期为" + mc(lastq2_y) + ",同比" + updown_q2[0] + str(round(growth_year[0],2)) + '%'
line2_q2 = "20Q2归母净利润" + mc(thisq2_g) + ",上年同期为" + mc(lastq2_g) + ",同比" + updown_q2[1] + str(round(growth_year[1],2)) + '%'
line3_q2 = "20Q2扣非归母净利润" + mc(thisq2_k) + ",上年同期为" + mc(lastq2_k) + ",同比" + updown_q2[2] + str(round(growth_year[2],2)) + '%'

In [35]:
print(line1_q2)
print(line2_q2)
print(line3_q2)

20Q2营业总收入2.84亿元,上年同期为2.85亿元,同比减少0.41%
20Q2归母净利润-735.19万元,上年同期为339.83万元,同比减少316.34%
20Q2扣非归母净利润-1580.99万元,上年同期为-741.02万元,同比减少113.35%
